In [1]:
import numpy as np
from util.whatwhere.encoder import *
from util.whatwhere.decoder import *
from util.whatwhere.plot import *
from util.willshaw.memory import *
from util.mnist.tools import *
from util.willshaw.plot import *
from util.pickleInterface import *
from util.pytorch.tools import np_to_grid
import wandb
from util.kldiv import *
from tqdm import trange

In [2]:
rng = np.random.RandomState(0)  # reproducible
K = 20
Q = 21
n_epochs = 5
b = 0.8
wta = True

In [3]:
Fs = 2
Tw = 0.9

In [4]:
imgs, lbls, _, _ = read_mnist(n_train=60000)

In [5]:
features = compute_features(imgs, lbls, K, Fs, rng, n_epochs, b)

In [6]:
codes, _ = compute_codes(
            imgs,
            K,
            Q,
            features,
            Tw,
            wta,
            n_epochs,
            b,
            Fs,
            set="trn",
        )

In [7]:
code_size = codes.shape[1]

In [8]:
wn = AAWN(code_size)

In [9]:
wn.store(codes)

In [10]:
ret = wn.retrieve(codes)

In [11]:
dp = csr_matrix.dot(codes, ret.T)

In [12]:
import matplotlib.pyplot as plt

In [13]:
def avg_bits_per_code(codes):
    nnz = codes.nnz
    num_codes = codes.shape[0]
    return nnz / num_codes

In [14]:
avg_bits_per_code(codes)

33.42928333333333

In [15]:
avg_bits_per_code(ret)

121.0372

In [16]:
hamming_distance_detailed(codes, ret)

(0.00993287037037037, 0.0, 0.00993287037037037)

In [17]:
def plot_discrete_hist(data, log=False):
    d = np.diff(np.unique(data)).min()
    left_of_first_bin = data.min() - float(d)/2
    right_of_last_bin = data.max() + float(d)/2
    plt.hist(data, np.arange(left_of_first_bin, right_of_last_bin + d, d))
    plt.yscale("log") if log else plt.yscale("linear")
    plt.show()

In [18]:
def subtr_max(arr):
    """ 
    subtracts the max of each row
    """
    for i in range(arr.shape[0]):
        m = np.max(arr[i])
        arr[i] -= m
    return arr

In [19]:
def plot_n_hits(arr, log=True):
    arr_sm = subtr_max(dp.toarray())
    n_max = np.count_nonzero(arr_sm==0, axis=1)
    plot_discrete_hist(n_max, log)


In [ ]:
plot_n_hits(dp)